# Challenges:
* links
* News
* Encode companies
* Encode people
* Encode cities and countries
* use text data embedding (word/document) or word freq 
* Remove unneeded columns (e.g. images,Deal announced on)
* Imputation
* impute year founded
* Twitter link: get followers and latest post?

# Notes on data
* Address col is useless , there's already city,state and country cols
* Crunchbase links are protected with captcha , can't be scraped

imports

In [ ]:
import pandas as pd
import numpy as np
import requests
import plotly.express as px

Reviewing a sample row from each file

In [19]:
acquired = pd.read_csv("Data/Acquired Tech Companies.csv")
acquired.iloc[0]

Company                                                     Day Software
CrunchBase Profile     http://www.crunchbase.com/organization/day-sof...
Image                  http://a5.images.crunchbase.com/image/upload/c...
Tagline                Day Software develops web applications that al...
Year Founded                                                         NaN
Market Categories                                               Software
Address (HQ)           Barfüsserplatz 6, Basel, Basel-Stadt, Switzerland
City (HQ)                                                          Basel
State / Region (HQ)                                          Basel-Stadt
Country (HQ)                                                 Switzerland
Description            Day was founded in Basel, Switzerland, in 1993...
Homepage                                              http://www.day.com
Twitter                                                              NaN
Acquired by                                        

In [20]:
acquiring = pd.read_csv("Data/Acquiring Tech Companies.csv")
acquiring.iloc[0]

Acquiring Company                                                                        Adobe
CrunchBase Profile                               www.crunchbase.com/organization/adobe-systems
Image                                        http://a2.images.crunchbase.com/image/upload/c...
Tagline                                      Adobe is an American multinational computer so...
Market Categories                            Photo Editing, Design, Creative, Software, Ima...
Year Founded                                                                              1982
IPO                                                                                       1986
Founders                                                         John Warnock, Charles Geschke
Number of Employees                                                                     11,144
Number of Employees (year of last update)                                               2012.0
Total Funding ($)                                 

In [21]:
acquisitions = pd.read_csv("Data/Acquisitions.csv")
acquisitions.iloc[0]

Acquisitions ID                                      EMC acquired Data Domain in 2009
Acquired Company                                                          Data Domain
Acquiring Company                                                                 EMC
Year of acquisition announcement                                                 2009
Deal announced on                                                           8/07/2009
Price                                                                  $2,100,000,000
Status                                                                    Undisclosed
Terms                                                                            Cash
Acquisition Profile                 http://www.crunchbase.com/acquisition/5dc676a1...
News                                                         EMC acquired Data Domain
News Link                           http://www.businesswire.com/news/home/20090708...
Name: 0, dtype: object

In [22]:
founders = pd.read_csv("Data/Founders and Board Members.csv")
founders.iloc[0]

Name                                                 Hans-Werner Hector
CrunchBase Profile      http://de.wikipedia.org/wiki/Hans-Werner_Hector
Role                                                            Founder
Companies                                                           SAP
Image                 http://images.forbes.com/media/lists/10/2006/4...
Name: 0, dtype: object

We will link between the files using these columns:
* Acquisitions ID to link the acquisitions
* 'Founders' and 'Name' to link the Founders

In [23]:
np.intersect1d(acquired.columns, acquisitions.columns).tolist()

['Acquisitions ID']

In [24]:
np.intersect1d(acquiring.columns, acquisitions.columns).tolist()

['Acquiring Company', 'Acquisitions ID']

In [9]:
def ValidateLink(url, timeout=15):
    session = requests.Session()
    # fake headers to make it seem like a real request
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7",
        "Accept-Language": "en-US,en;q=0.9",
        "Accept-Encoding": "gzip, deflate, br",
        "Connection": "keep-alive",
        "Upgrade-Insecure-Requests": "1",
        "DNT": "1",
    }
    session.headers.update(headers)
    try:
        response = session.get(url, timeout=timeout, allow_redirects=True, stream=True)
        status_code = response.status_code
        response.close()
        if status_code < 400:
            return True
        else:
            return False
    except Exception as e:
        return False

In [ ]:
def ValidateLinks(urls):
    results = []
    for url in urls:
        results.append(ValidateLink(url))
        if results[-1]:
            return results
    return results

In [ ]:
def ValidateLinksDF(df):
    for col in df.columns:
        for val in df[col]:
            if type(val) == str and ("http" in val):
                print(col)
                results = ValidateLinks(df[col])
                if not pd.Series(results).any():
                    print(f'Column "{col}" had no valid links , or is using captcha.')
                    print("Try it yourself:")
                    print(df[col][0] + "\n")
                break

In [12]:
ValidateLinksDF(acquired)

CrunchBase Profile
Column "CrunchBase Profile" had no valid links , or is using captcha.
Try it yourself:
http://www.crunchbase.com/organization/day-software

Image
Column "Image" had no valid links , or is using captcha.
Try it yourself:
http://a5.images.crunchbase.com/image/upload/c_pad,h_500,w_500/v1397187256/0ef62e4243274cafe1a563dda2bca363.png

Homepage
Twitter
API


* CrunchBase is using CAPTCHA , so we won't drop it now but we will proccess it later
* Image links are all corrupt so we will drop the column 

In [25]:
acquired = acquired.drop("Image", axis=1)

In [13]:
ValidateLinksDF(acquiring)

Image
Column "Image" had no valid links , or is using captcha.
Try it yourself:
http://a2.images.crunchbase.com/image/upload/c_pad,h_500,w_500/v1397180657/2cd912e176145af3618549d60b7959a1.png

Homepage
Twitter
API


* drop Image also

In [26]:
acquiring = acquiring.drop("Image", axis=1)

In [14]:
ValidateLinksDF(acquisitions)

Acquisition Profile
Column "Acquisition Profile" had no valid links , or is using captcha.
Try it yourself:
http://www.crunchbase.com/acquisition/5dc676a13d41c2ee87169ce59476ec2d

News Link


* acquisitions profile is also a crunchbase link

In [15]:
ValidateLinksDF(founders)

CrunchBase Profile
Image


We don't need the exact address of the company, we already have the city , state and country

In [27]:
acquired=acquired.drop('Address (HQ)', axis=1)
acquiring=acquiring.drop('Address (HQ)', axis=1)

Dropping 'year of last update' of the number of employees , because we don't need it directly and can't use it in any way to pridct the current number

In [28]:
acquiring=acquiring.drop('Number of Employees (year of last update)', axis=1)

In [29]:
acquiring.iloc[12]['Tagline']

'EMC manufactures information management and storage software systems.'

In [30]:
for l in acquiring.iloc[12]['Description'].split('.'):
    print(l +'\n') 

EMC is a global leader in enabling businesses and service providers to transform their operations and deliver information technology as a service ITaaS

 Fundamental to this transformation is cloud computing

 Through innovative products and services, EMC accelerates the journey to cloud computing, helping IT departments to store, manage, protect and analyze their most valuable asset — information — in a more agile, trusted and cost-efficient way

EMC is a manufacturer of software and systems for information management and storage

  EMC produces a range of enterprise storage products, including hardware disk arrays and storage management software

 Its flagship product, the Symmetrix, is the foundation of storage networks in many large data centers





* 'Tagline' contains a brief and precise description of the company , while the 'Description' is very long and doesn't provide any more important details, 
so we will drop the 'Description'

In [31]:
acquiring=acquiring.drop('Description', axis=1)
acquired=acquired.drop('Description', axis=1)

**Adding the target variable**

In [32]:
acquisitions['Price']=[int(price.removeprefix('$').replace(',','')) for price in acquisitions['Price']]

In [34]:
fig = px.scatter(
    acquisitions,
    x='Year of acquisition announcement',
    y='Price',
    title='Acquisition Price by Year'
)
fig.show()

There was a wrongly entered value, so I looked at the link and corrected it

In [35]:
acquisitions.loc[acquisitions['Year of acquisition announcement']==2104,'Year of acquisition announcement']=2014 

In [36]:
acquired['Price']=None
acquired['Year of acquisition announcement']=None

In [37]:
for i,company in enumerate(acquisitions['Acquired Company']):
    acquired.loc[acquired['Company']==company,'Price']=acquisitions.iloc[i]['Price']
    acquired.loc[acquired['Company']==company,'Year of acquisition announcement']=acquisitions.iloc[i]['Year of acquisition announcement']

update the datatypes automatically

In [38]:
acquired=acquired.infer_objects()
acquisitions=acquisitions.infer_objects()

In [39]:
acquired.describe()

,Year Founded,Price,Year of acquisition announcement
count,241.000000,3.100000e+02,310.000000
mean,2000.526971,1.067965e+09,2008.961290
std,14.027365,3.529433e+09,4.238648
min,1840.000000,1.400000e+06,1995.000000
25%,1998.000000,6.500000e+07,2007.000000
50%,2003.000000,1.780000e+08,2010.000000
75%,2007.000000,5.787500e+08,2012.000000
max,2014.000000,4.500000e+10,2014.000000


The specific date which the deal was announced on doesn't matter , what matters is the year so the model can know that inflation affects the price

In [40]:
acquisitions=acquisitions.drop('Deal announced on',axis=1)

Plotting again without the error, now we can see that the overall trend of prices tends to go up, that's why we added the 'Year of acquisitions announcement' column

In [41]:
acquired.iloc[0]

Company                                                                  Day Software
CrunchBase Profile                  http://www.crunchbase.com/organization/day-sof...
Tagline                             Day Software develops web applications that al...
Year Founded                                                                      NaN
Market Categories                                                            Software
City (HQ)                                                                       Basel
State / Region (HQ)                                                       Basel-Stadt
Country (HQ)                                                              Switzerland
Homepage                                                           http://www.day.com
Twitter                                                                           NaN
Acquired by                                                                     Adobe
Acquisitions ID                           Adobe System

In [42]:
fig = px.scatter(
    acquired,
    x='Year Founded',
    y='Price',
    title='Acquisition Price by Year'
)
fig.show()

Another error found and corrected

In [43]:
acquired.loc[acquired['Year Founded']==1840,'Year Founded']=2006
acquired.loc[acquired['Year Founded']==1933,'Year Founded']=1989

In [44]:
fig = px.scatter(
    acquired,
    x='Year Founded',
    y='Price',
    title='Acquisition Price by Year'
)
fig.show()

In [45]:
fig = px.scatter(
    acquired,
    x='Year of acquisition announcement',
    y='Price',
    title='Acquisition Price by Year'
)
fig.show()

In [60]:
acquired.iloc[5]

Company                                                                         Alexa
CrunchBase Profile                       http://www.crunchbase.com/organization/alexa
Tagline                             Alexa, a subsidiary of Amazon, is a web inform...
Year Founded                                                                   1996.0
Market Categories                                                         Curated Web
City (HQ)                                                               San Francisco
State / Region (HQ)                                                        California
Country (HQ)                                                            United States
Homepage                                                         http://www.alexa.com
Twitter                                              http://twitter.com/AlexaInternet
Acquired by                                                                    Amazon
Acquisitions ID                                       

### There isn't any new useful information that we can get out of those , so we will drop them

* "CrunchBase Profile" and "API" columns are both on the crunchbase website , which uses captcha so we can't scrap it, and their API is paid , and the provided API key is invalid , so we can't use it
* "Homepage" column contains the link to the website of every company , and they aren't all the same so we can't apply a function or a program to extract certain information about them. To use the link , this would require us to go over into each of them one by one , which isn't  feasible
* "Twitter"


In [ ]:
acquired=acquired.drop(['CrunchBase Profile','Homepage','Twitter','Acquisition ID','API'])

In [68]:
acquired.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 310 entries, 0 to 309
Data columns (total 15 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Company                           310 non-null    object 
 1   CrunchBase Profile                310 non-null    object 
 2   Tagline                           307 non-null    object 
 3   Year Founded                      241 non-null    float64
 4   Market Categories                 287 non-null    object 
 5   City (HQ)                         275 non-null    object 
 6   State / Region (HQ)               273 non-null    object 
 7   Country (HQ)                      276 non-null    object 
 8   Homepage                          273 non-null    object 
 9   Twitter                           168 non-null    object 
 10  Acquired by                       309 non-null    object 
 11  Acquisitions ID                   310 non-null    object 
 12  API     

In [65]:
import snscrape

In [81]:
key= ''
secret = ''
bearer= 'AAAAAAAAAAAAAAAAAAAAADQA0wEAAAAAZcRLwct53Q05%2FR7Bq45uHKj0Sfk%3DoqjlsnWRjjh7KjfNJADmstZFG9B7tH1cc7Tah6auxih6vfKdUQ'

In [118]:
import tweepy
import os

from dotenv import load_dotenv

load_dotenv()  
def get_followers_twitter_api(username):
    client = tweepy.Client(bearer_token=os.getenv('TWITTER_API_BEARER'))
    user = client.get_user(username=username, user_fields=['public_metrics'])
    return user.data.public_metrics['followers_count']


In [121]:
get_followers_twitter_api('elonmusk')

TooManyRequests: 429 Too Many Requests
Too Many Requests

In [102]:
import requests
from bs4 import BeautifulSoup
import re

def get_follower_count(username):
    url = f"https://twitter.com/{username}"
    headers = {
            "User-Agent": (
                "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                "AppleWebKit/537.36 (KHTML, like Gecko) "
                "Chrome/122.0.0.0 Safari/537.36"
            ),
            "Accept-Language": "en-US,en;q=0.9",
            "Accept-Encoding": "gzip, deflate, br",
            "Referer": "https://www.google.com/",
            "Connection": "keep-alive",
            "DNT": "1",
        }
    response = requests.get(url,headers=headers)
    soup = BeautifulSoup(response.text, "html.parser")

    # Look for follower count in the %%HTM
    return soup
    match = re.search(r'"followers_count":(\d+)', soup.text)
    if match:
        return int(match.group(1))
    else:
        return None


In [103]:
get_follower_count('elonmusk')

<!DOCTYPE html>
<html dir="ltr" lang="en"><head><meta charset="utf-8"/><meta content="width=device-width,initial-scale=1,maximum-scale=1,user-scalable=0,viewport-fit=cover" name="viewport"/><link href="//abs.twimg.com" rel="preconnect"/><link href="//abs.twimg.com" rel="dns-prefetch"/><link href="//api.twitter.com" rel="preconnect"/><link href="//api.twitter.com" rel="dns-prefetch"/><link href="//api.x.com" rel="preconnect"/><link href="//api.x.com" rel="dns-prefetch"/><link href="//pbs.twimg.com" rel="preconnect"/><link href="//pbs.twimg.com" rel="dns-prefetch"/><link href="//t.co" rel="preconnect"/><link href="//t.co" rel="dns-prefetch"/><link href="//video.twimg.com" rel="preconnect"/><link href="//video.twimg.com" rel="dns-prefetch"/><link as="script" crossorigin="anonymous" href="https://abs.twimg.com/responsive-web/client-web/vendor-7940b00b.e74f2a9a.js" nonce="NGVlZWE4MDUtNjE1Yi00MTYyLThhNDgtM2FhMGRlZmRkMTk2" rel="preload"/><link as="script" crossorigin="anonymous" href="https:/

In [115]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time

def get_twitter_followers(username):
    options = Options()
    # options.headless = False
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")

    driver = webdriver.Chrome()
    url = f"https://twitter.com/{username}"
    driver.get(url)

    time.sleep(5)  # wait for page to load

    try:
        # Find followers element by label
        followers_element = driver.find_element(By.XPATH, "//a[contains(@href,'/followers')]/span[1]/span")
        followers = followers_element.text
    except Exception as e:
        followers = None
        print(f"Error: {e}")

    driver.quit()
    return followers


In [116]:
get_twitter_followers('elonmusk')

KeyboardInterrupt: 

In [70]:


import asyncio
from playwright.sync_api import sync_playwright


def scrape_profile(url: str) -> dict:
    """
    Scrape a X.com profile details e.g.: https://x.com/Scrapfly_dev
    """
    _xhr_calls = []

    def intercept_response(response):
        """capture all background requests and save them"""
        # we can extract details from background requests
        if response.request.resource_type == "xhr":
            _xhr_calls.append(response)
        return response

    with sync_playwright() as pw:
        browser = pw.chromium.launch(headless=False)
        context = browser.new_context(viewport={"width": 1920, "height": 1080})
        page = context.new_page()

        # enable background request intercepting:
        page.on("response", intercept_response)
        # go to url and wait for the page to load
        page.goto(url)
        page.wait_for_selector("[data-testid='primaryColumn']")

        # find all tweet background requests:
        tweet_calls = [f for f in _xhr_calls if "UserBy" in f.url]
        for xhr in tweet_calls:
            data = xhr.json()
            return data['data']['user']['result']



if __name__ == "__main__":
    print(asyncio.run(scrape_profile("https://x.com/Scrapfly_dev")))



ModuleNotFoundError: No module named 'playwright'

In [67]:
import snscrape.modules
import snscrape.modules.twitter


snscrape.modules.twitter.TwitterUserScraper('elonmask')

AttributeError: 'FileFinder' object has no attribute 'find_module'

In [ ]:

def get_twitter_followers(username):
    try:
        user = sntwitter.TwitterUserScraper(username).entity
        return user.followersCount
    except Exception as e:
        print("Error:", e)
        return None
